# Lab 3: Gestión de ciclo de vida de un índice




## 1. Políticas de gestión de ciclo de vida de un índice.

Puesto que la creación de una política para gestionar el ciclo de vida de un índice requiere realizar una serie de operaciones de forma conjunta lo vamos a ver en detalle y paso a paso.

Definir una buena política para gestión de la vida de un índice requiere entender y las necesidades del caso de uso para el que se están utilizando los índices para adecuarse de forma correcta a sus necesidades. 

En el ejemplo que estamos implementando queremos estructurar la información recogida en los logs de nuestros servidores web e indexarla estructurada en documentos en elasticsearch para explotar esta información y poder por un lado aplicar politicas de calidad de servicio, detectar ataques de seguridad y de denegación de servico.

Teniendo el caso de uso en mente, lo primero que tenemos que analizar es la naturaleza de nuestros datos. En este caso, los datos que vamos a recibir son líneas de log de los servidores web que contienen información sobre las peticiones que se realizan a dicho servidor con su marca de tiempo. Por lo que podemos tomar cada línia de log como un evento realizado en un momento en el tiempo. 

El segundo punto a tener en cuenta, es el tiempo de validez de nuestros datos y del resultado del análisis que de ellos agamos. Por ejemplo, si queremos detectar un ataque de denagación de servicio en tiempo real, la valided de la analítica es cercana al minuto, si de lo que estamos hablando de de analizar el tráfico para mejorar la calidad del servicio, la validez de la analítica está en torno a la hora, etc.

Todos estos datos nos indicarán la disponibiliad y validez de dato:

<img src="../../images/els/tiempoDeVidaDato.png" alt="index management"/>


Otro aspecto a tener en cuenta es el volumen de datos, según se van ingestando mas líneas de log, el volumen de datos a ingestar y retener aumenta significativamente y por tanto el coste de almacenamiento. Por lo que es necesario adecuar la disponibilidad y el tiempo de retención del dato a las necesidades del caso de uso. 

El dato mas disponible es el más caro, por lo que no interesa sólo tener el volumen de datos necesario y durante el tiempo de validez del dato para la analítica del caso de uso, e ir rotando a niveles de acceso mas económicos hasta definir cual es su momento de archivado y posterior borrado (borrar mucahs veces significa pasar el dato a otro tipo de almacenamiento en otro data store pensado para almacenar grandes volúmenes de datos a muy bajo coste, pero con una latencia de acceso al dato de incluso días).

Elasticserch nos permite definir "Data Tiers". Cada Data Tier es un conjunto de nodos que tienen unas características comunes en cuanto a hardware. Esto es muy útil por ejemplo, cuando se instala Elasticsearch en un proveedor de infraestructura en la nuve, pudiendo definir nodos mas caros o mas baratos en función de su hardware, asignarlos a un tier, para posteriormente alojar los índices de cada nivel del ciclo de vida en el tier adecuado según sus necesidades. De esta forma podemos controlar el coste de la infraestructura.

<img src="../../images/els/DataTiers.png" alt="index management"/>

Elasticsearch nos permite por un lado identificar y definir diferentes índices para cada uno de los niveles del dato, un mecanismo de rotación de datos entre índices basado en tiempo y/o en volumen de datos almacenado y un poceso de archivado y borrado. Todo esto se orquesta y automatiza definiendo una política de gestión del ciclo de vida de un índice. 


### 2.1. Definición de la política

Para definir la política, lo primero que tenmos que hacer es definir los niveles del ciclo de vida de un índice. Elasticserach define 5 niveles:

* **Hot:** El nivel por defecto y por lo tanto obligatorio y se utiliza para alojar los datos más actuales. Se pueden añadir, borrar, modificar y consultar documentos.
* **Warm:** El índice es de sólo lectura pero se puede seguir consultando. Contiene documentos que no se suelen modificar, pero que se necesita seguir consultando.
* **Cold:** El índice es de sólo lectura y aloja documentos que no se suelen consultar con mucha frecuencia.
* **Frozen:** Igual que el nivel anterior, pero la frecuencia de acceso es aún menor y el tiempo de acceso a los documentos es mayor (menor disponibilidad del dato).
* **Delete:** El índice se borra ya que los documentos se consideran que no tienen validez o no son útiles.

En Elasticsearch la unidad mínima de gestión de datos es el índice, por lo que cada uno de estos niveles estará compuesto por uno o varios índices.

Y una vez identificados, es necesario definir las acciones que se tienen que producir para rotar los datos de un nivel a otro. Para pasar los datos de un nivel a otro lo que se realiza es una operación de rotación de indíces o rollover.

Vamos a ver los diferentes conceptos que nos proporciona Elasticaserch para construir una política en base a lo definido previamente:

* **Bootstrap index**: Es el índice inicial que hay que crear manualmente y en el que se empezarán a añadir los datos de nuestro caso de uso. Constituirá el primer índice de escritura y por tanto el primer índice del nivel Hot.
* **Write alias**: Los clientes de Elasticsearch no tienen por que saber cuál es el índice activo o sobre el que pueden añadir mas datos. Para interactuar con el índice de escrutura activo actual se utiliza un alias de índice, que es el nombre de índice que utilizarán los clientes y que la política de gestión se encargará de asignar a los nuevos índices de escritura.
* **Index rollover**: Un nuevo índcie de escritura es creado y se le asigna el alias de escritura. Al anerior índice se le asigna permisos de sólo lectura. Esta operación se realiza cuando el índice actual de escritura alcanza un tamaño máximo de datos o de documentos especificado o alcanza su edad máxima, es decir el tiempo máximo que puede estar activo un índice para escritura.

La escritura siempre se realiza sobre el índice activo de escritura a través del alias y la lectura se puede ejecutar sobre dodos los índices usando un wildcard:

<img src="../../images/els/ilmEscrituraLectura.png" alt="index management"/>

### 2.2. Implementación de la política

Vamos a ver los pasos a ejecutar para definir la política a través de la API de Elasticsearch:

### Paso 1: Definir los niveles y las condiciones para el rollover entre niveles:

Nota: No todos los niveles y opciones están disponibles en la licencia gratuita. Para este ejercicio sólo vamos a utilizar los niveles disponibles en la licencia gratuita.

Para definir la política realizaremos la apetición _ilm/policy/policy_name: 

`
PUT _ilm/policy/web-logs-policy
{
    "policy": {
        "phases": {
            "hot": {
                "actions": {
                    "rollover": {
                        "max_age": "1h",
                        "max_size": "50gb"
                    }
                }
            },
            "warm": {
                "min_age": "1d",
                "actions": {}
            },
            "cold": {
                "min_age": "30d",
                "actions": {}
            },
            "delete": {
                "min_age": "365d",
                "actions": {
                  "delete": {}
                }
            }
        }
    }
}
`

Esta política ejecuta las siguientes acciones:
* El índice del nivel "hot" se rotará en el mismo nivel, cuando alcance un tamaño de 50gb o tenga una antiguedad de 1 hora, lo que ocurra primero. 
* Una vez pasado un día desde que el índice se rotó en "hot", el índcie hot se pasará al nivel "warm".
* Una vez pasados 30 días desde que el índice entró en el nivel "warm", se pasará al nivel cold.
* Una vez pasados 365 días (1 año), el índice se borrará.

En cada nivel se pueden definir diferentes acciones:
* **allocate**: permite indicar en que nodos alojar las particiones del índice así como cambira el número de particiones y réplicas del índice. No se puede usar en el nivel "hot".
* **force_merge**: indica el número de segmentos que queremos que tenga el índice al ser rotado. Es necesario definir la acción de rollover. Convierte el índice en sólo lecutra. Valido para las fases "hot" y "warm".
* **migrate**: Mueve los shards del índcie al data tier especificado.

### Paso 2: Crear el índex template para los nuevos índices

Puesto que cada vez que se rota el índice activo de escritura se crea un nuevo índice, es necesario crear un index template para crearlo con el esquema necesario. 
Para simplificar, y puesto que ya hemos visto como crear templates, sólo vamos a definir los campos necesarios para la creación de politicas.

`
PUT _index_template/web-logs
{
    "index_patterns": ["web-logs-*"],
    "template": {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1,
            "index.lifecycle.name": "web-logs-policy",
            "index.lifecycle.rollover_alias": "web-logs"
        },
        "mappings": {
          "dynamic_templates": [
            {
              "strings_as_keyword": {
                "mapping": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "match_mapping_type": "string"
              }
            }
          ],
          "date_detection": false,
          "properties": {        
            "log": {
              "properties": {
                "file": {
                  "properties": {
                    "path": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    }
                  }
                }
              }
            },
            "source": {
              "properties": {
                "address": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "ip": {
                  "type": "ip"
                },
                "geo": {
                  "properties": {
                    "region_iso_code": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "continent_name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "city_name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "country_iso_code": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "country_name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "region_name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "location": {
                      "type": "geo_point"
                    }
                  }
                },
                "as": {
                  "properties": {
                    "number": {
                      "type": "long"
                    },
                    "organization": {
                      "properties": {
                        "name": {
                          "ignore_above": 1024,
                          "fields": {
                            "text": {
                              "norms": false,
                              "type": "text"
                            }
                          },
                          "type": "keyword"
                        }
                      }
                    }
                  }
                }
              }
            },
            "url": {
              "properties": {
                "original": {
                  "ignore_above": 1024,
                  "type": "keyword",
                  "fields": {
                    "text": {
                      "norms": false,
                      "type": "text"
                    }
                  }
                }
              }
            },
            "apache": {
              "properties": {
                "access": {
                  "properties": {
                    "ssl": {
                      "properties": {
                        "cipher": {
                          "ignore_above": 1024,
                          "type": "keyword"
                        },
                        "protocol": {
                          "ignore_above": 1024,
                          "type": "keyword"
                        }
                      }
                    }
                  }
                },
                "error": {
                  "properties": {
                    "module": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    }
                  }
                }
              }
            },
            "@timestamp": {
              "type": "date"
            },
            "http": {
              "properties": {
                "request": {
                  "properties": {
                    "referrer": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "method": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    }
                  }
                },
                "response": {
                  "properties": {
                    "status_code": {
                      "type": "long"
                    },
                    "body": {
                      "properties": {
                        "bytes": {
                          "type": "long"
                        }
                      }
                    }
                  }
                },
                "version": {
                  "ignore_above": 1024,
                  "type": "keyword"
                }
              }
            },   
            "event": {
              "properties": {
                "ingested": {
                  "type": "date"
                },
                "outcome": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "original": {
                  "type": "text"
                },
                "created": {
                  "type": "date"
                },
                "kind": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "category": {
                  "ignore_above": 1024,
                  "type": "keyword"
                }
              }
            },
            "user": {
              "properties": {
                "name": {
                  "ignore_above": 1024,
                  "type": "keyword",
                  "fields": {
                    "text": {
                      "norms": false,
                      "type": "text"
                    }
                  }
                }
              }
            },
            "user_agent": {
              "properties": {
                "original": {
                  "ignore_above": 1024,
                  "type": "keyword",
                  "fields": {
                    "text": {
                      "norms": false,
                      "type": "text"
                    }
                  }
                },
                "os": {
                  "properties": {
                    "name": {
                      "ignore_above": 1024,
                      "type": "keyword",
                      "fields": {
                        "text": {
                          "norms": false,
                          "type": "text"
                        }
                      }
                    },
                    "version": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    },
                    "full": {
                      "ignore_above": 1024,
                      "type": "keyword",
                      "fields": {
                        "text": {
                          "norms": false,
                          "type": "text"
                        }
                      }
                    }
                  }
                },
                "name": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "version": {
                  "ignore_above": 1024,
                  "type": "keyword"
                },
                "device": {
                  "properties": {
                    "name": {
                      "ignore_above": 1024,
                      "type": "keyword"
                    }
                  }
                }
              }
            }
          }
        }
      }
}
`

Estamos añadiendo dos settings nuevos:
* **index.lifecycle.name**: El nombre de la política que acabmos de definir, para saber con que patrón es necesario crear los nuevos índcies al ser rotados.
* **index.lifecycle.rollover_alias**: El alias de escritura que se le asignará al nuevo índice creado después del rotado.

### Paso 3: Creamos el bootstrap index

Como hemos visto antes, el índice inicial es necesario crearlo a mano.
Para simplificar, y puesto que ya hemos visto como crear índices, sólo vamos a definir los campos necesarios para la creación de politicas.

`
PUT web-logs-000001
{
    "aliases": {
        "web-logs":{
            "is_write_index": true
        }
    }
}
`

* El nombre del índice tiene que seguir el siguiente patrón `^.*-\d+$`es decir que termine en dígito. De esta manera la política sabrá como incrementar el numero de índice de forma automática.
* Es necesario asignarle el álias de escritura y a la vez indicarle que el índcie es de escritura puesto que va a ser el primer índice de escritura activo.

### Paso 4: Utilizar el índice con normalidad

Para la escritura utilizaremos siempre el álias:

`
POST web-logs/_doc
{
    "asset": "motor 123",
    "timestamp": "2021-02-14T13:00:00.000Z",
    "metric": "temperature",
    "value": 24.5
}
`

y para la consulta lo realizaremos sobre todos los índices usando un wildcard:

`
GET web-logs*/_search
`

Por último, siempre podemos forzar un rollover:

`
POST web-logs/_rollover
`
